<a href="https://colab.research.google.com/github/Asynktra/brain/blob/main/brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi   # Check GPU (you want Tesla T4, A100, or V100 ideally)
!pip install torch transformers datasets accelerate bitsandbytes


/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
BASE_DIR = "/content/drive/MyDrive/brain_project"  # persistent
!mkdir -p $BASE_DIR/checkpoints


In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Load dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

# Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

# Model
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training setup (compatible with older transformers)
args = TrainingArguments(
    output_dir=f"{BASE_DIR}/checkpoints/phase1_wiki",
    # removed evaluation_strategy / eval_steps for compatibility with older transformers
    save_steps=500,
    logging_steps=100,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()
model.save_pretrained(f"{BASE_DIR}/checkpoints/phase1_wiki_best")
tokenizer.save_pretrained(f"{BASE_DIR}/checkpoints/phase1_wiki_best")


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

/tmp/ipython-input-234185306.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.890300


In [ ]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

# ✅ Load dataset properly (parquet export, not old script)
dialogue_ds = load_dataset("daily_dialog", "plain_text")

def dialog_tokenize(batch):
    # Join utterances in a single string
    joined = [" ".join(d) for d in batch["dialog"]]
    return tokenizer(joined, truncation=True, padding="max_length", max_length=128)

# Tokenize
tokenized_dialogue = dialogue_ds.map(
    dialog_tokenize,
    batched=True,
    remove_columns=dialogue_ds["train"].column_names
)

# Load Phase 1 model
model = AutoModelForCausalLM.from_pretrained(f"{BASE_DIR}/checkpoints/phase1_wiki_best")

# Training args (fix evaluation_strategy if your version complains)
args2 = TrainingArguments(
    output_dir=f"{BASE_DIR}/checkpoints/phase2_dialog",
    eval_strategy="steps",   # ✅ if "evaluation_strategy" breaks, use "eval_strategy"
    eval_steps=200,
    save_steps=200,
    logging_steps=50,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

# Trainer
trainer2 = Trainer(
    model=model,
    args=args2,
    train_dataset=tokenized_dialogue["train"],
    eval_dataset=tokenized_dialogue["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train
trainer2.train()

# Save
model.save_pretrained(f"{BASE_DIR}/checkpoints/phase2_dialog_best")
tokenizer.save_pretrained(f"{BASE_DIR}/checkpoints/phase2_dialog_best")


In [ ]:
import torch

def apply_mood_bias(logits, tokenizer, mood="flirty", strength=2.0):
    """
    Nudges the model's output distribution toward certain mood-related words.

    Args:
        logits: torch.Tensor of shape [batch, seq_len, vocab_size]
        tokenizer: Hugging Face tokenizer
        mood (str): "flirty", "sarcastic", "quirky", etc.
        strength (float): how strongly to bias toward mood words

    Returns:
        torch.Tensor: modified logits
    """
    mood_bias = {
        "flirty": ["love", "darling", "kiss", "cute"],
        "sarcastic": ["yeah", "sure", "obviously", "genius"],
        "quirky": ["banana", "chaos", "weird", "sparkle"]
    }

    words = mood_bias.get(mood, [])
    if not words:
        return logits

    # Work on a clone to avoid in-place modifications messing with gradients
    biased_logits = logits.clone()

    for w in words:
        tok_ids = tokenizer.encode(w, add_special_tokens=False)
        if tok_ids:  # Some words might tokenize into multiple subwords
            for tid in tok_ids:
                if tid < biased_logits.size(-1):  # valid vocab id
                    biased_logits[0, -1, tid] += strength

    return biased_logits


In [ ]:
import torch
from transformers import GPT2Tokenizer, AutoModelForCausalLM, LogitsProcessor, LogitsProcessorList
import os

# Mood bias processor
class MoodBiasProcessor(LogitsProcessor):
    def __init__(self, tokenizer, mood="quirky", strength=2.0):
        self.tokenizer = tokenizer
        self.mood = mood
        self.strength = strength
        self.mood_bias = {
            "flirty": ["love", "darling", "kiss", "cute"],
            "sarcastic": ["yeah", "sure", "obviously", "genius"],
            "quirky": ["banana", "chaos", "weird", "sparkle"]
        }

    def __call__(self, input_ids, scores):
        words = self.mood_bias.get(self.mood, [])
        for w in words:
            tok_ids = self.tokenizer.encode(w, add_special_tokens=False)
            for tid in tok_ids:
                if tid < scores.size(-1):
                    scores[:, tid] += self.strength
        return scores

# Load checkpoint (phase2 if exists, else phase1)
if os.path.exists(f"{BASE_DIR}/checkpoints/phase2_dialog_best"):
    ckpt = f"{BASE_DIR}/checkpoints/phase2_dialog_best"
else:
    ckpt = f"{BASE_DIR}/checkpoints/phase1_wiki_best"

tokenizer = GPT2Tokenizer.from_pretrained(ckpt)
model = AutoModelForCausalLM.from_pretrained(ckpt)

def chat(prompt, mood="quirky", max_new_tokens=60):
    processor = MoodBiasProcessor(tokenizer, mood=mood, strength=2.0)
    logits_processors = LogitsProcessorList([processor])

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        logits_processor=logits_processors
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(chat("Hey, how are you?", mood="flirty"))
